In [ ]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
import math
import time

In [ ]:
with open('generated/melodyData2.txt', 'r') as f:
    parts = eval(f.read())
    print ("Number of parts:" + str(len(parts)))

In [ ]:
# number of parts to train on
train_all_parts = True
num_data = int(1e9) if train_all_parts else 250
# max length of each part
max_sequence_length = int(1e9) if train_all_parts else 300 

# one-hot encoding
encodings = {}
encodingIndex = 0
for part in parts[:num_data]:
    for note in part[:max_sequence_length]:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

# decoder constructed by reversing one-hot encoding
decodings = {}
for k, v in encodings.items():
    decodings[v] = k

# encode everything in a
data_encoded = []
for part in parts[:num_data]:
    data_encoded.append([encodings[note] for note in part[:max_sequence_length]])

num_data = min(num_data, len(data_encoded))


In [ ]:
# one-hot encode the data
def generate_data(data_encoded, encodings):
    X = []
    Y = []
    # given data_encoded, generate training data by looping 
    for i in range(len(data_encoded)):
        currentX = []
        currentY = []
        for j in range(len(data_encoded[i])-1):
            currentX.append(data_encoded[i][j])
            currentY.append(data_encoded[i][j+1])
        X.append(currentX)
        Y.append(currentY)

    X_onehot = []
    for seq in X:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
        X_onehot.append(onehot)
    X = X_onehot
    
    Y_onehot = []
    for seq in Y:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
            
        Y_onehot.append(onehot)
    Y = Y_onehot

    return X, Y

X_train, Y_train = generate_data(data_encoded[:math.floor(num_data*0.7)], encodings)
#X_test, Y_test = generate_data(data_encoded[10+math.floor(num_data*0.7)], encodings)
X_test, Y_test = generate_data(data_encoded[math.floor(num_data*0.7):], encodings)

# pads sequences so we can convert to numpy arrays
X_train = tf.keras.utils.pad_sequences(X_train, padding='pre')
Y_train = tf.keras.utils.pad_sequences(Y_train, padding='pre')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Masking(mask_value=0, batch_input_shape=(1, None, len(encodings))),
    tf.keras.layers.LSTM(64, stateful=True, return_sequences=True),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

timeTaken = time.time()

# Train the model one time step at a time
accuracy = 0
epochs = 2
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    total_loss = 0
    for i, sequence in enumerate(X_train):
        # Reset states at the beginning of each sequence
        model.reset_states() 
        x = sequence.reshape((1, sequence.shape[0], len(encodings)))
        y = Y_train[i].reshape((1, sequence.shape[0], len(encodings)))
        loss, note_accuracy = model.train_on_batch(x, y)
        total_loss += loss
    accuracy += note_accuracy
    print(f"Accuracy {accuracy/(epoch+1)}, Loss {total_loss/len(X_train)}")
    print(f"Epoch time {time.time() - timeTaken}")
    timeTaken = time.time()
# 6.27 seconds for 5 epochs
# 1000 seconds for 1 epoch (full data)

In [ ]:
from datetime import datetime
model.save_weights(f"./checkpoints/model_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.h5")

In [ ]:
test_parts = 30
# tests model accuracy

correct = 0.0
total = 0


for i in range(min(test_parts, len(X_test))):
    # reset model
    model.reset_states()
    print (i)
    
    for j in range(len(X_test[i])):
        # use predict_on_batch
        
        curr_note = X_test[i][j].reshape(1, 1, X_test[i][j].shape[0])
        pred = model.predict_on_batch(curr_note)
        
        if (np.argmax(pred) == np.argmax(Y_test[i][j])):
            correct += 1
        total += 1
    # replace this, use train_on
print(correct/total)